In [1]:
# Mengimpor library untuk preprocessing data
import csv
import pandas as pd
import numpy as np
import string
import nltk
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
pd.set_option('display.max_colwidth',100)
     

In [2]:
#Membuat fungsi load_data untuk membuka file csv hasil crawling data
def load_data():
    data = pd.read_csv('D:/hasilSentimen2/Dataset.csv',encoding='ISO-8859-1')
    return data

In [3]:
#Memasukkan kedalam variabel
tweet_df = load_data()
#Menampilkan 500 data teratas
tweet_df.head(500)

,Unnamed: 0,Tanggal,Pengguna,Tweet,Sentimen
0,0,12/12/2022 04:19,ENS53311139,YTH KAPOLRI DENSUS BNPT STOP MENDISCREDIT ISLAM DGN ISIS TERORIS RADIKAL INTOLERANSI KARENA BUKA...,Netral
1,1,12/12/2022 04:16,RawasUluSektor,Bhabinkamtibmas Polsek Rawas Ulu Polres Muratara Polda Sumsel menyebarkan maklumat Kapolri tenta...,Netral
2,2,12/12/2022 04:14,sitohang76,Indahnya toleransi Indonesia CC yth Bapak Menkopolhukam Bapak Kapolri Bapak Presiden Saya percay...,Positif
3,3,12/12/2022 04:11,polsek_gemolong,Resepsi Kaesang Erina Berjalan Sukses Kapolri Apresiasi Masyarakat Surakarta Kapolri Apresiasi M...,Positif
4,4,12/12/2022 04:07,RawasUluSektor,Bhabinkamtibmas Polsek Rawas Ulu Polres Muratara Polda Sumsel menyebarkan maklumat Kapolri tenta...,Netral
...,...,...,...,...,...
495,495,10/12/2022 12:47,JuarsyahZed,POLSEK MESUJI MAKMUR ulurkan tangan mu untuk membantu sesama untuk memberikan manfaat bagi orang...,Netral
496,496,10/12/2022 12:47,HUMAS_4lawang,Arahan Bapak kapolri kepada jajaran untuk meraih kembali kepercayaan publik hingga menghindari p...,Netral
497,497,10/12/2022 12:46,JuarsyahZed,POLSEK MESUJI MAKMUR Tebarkan senyuman dengan penuh ke ikhlasan dalam setiap langkah mabespolri ...,Positif
498,498,10/12/2022 12:46,Desri24428503,Cega Pak knp kinerja kapolri ini tdk di kontrol Knp dbiarkan sj seenaknya spt itu tdk tg jwb di ...,Netral


In [4]:
#Mneghitung jumlah sentimen
tweet_df["Sentimen"].value_counts()

Netral     242
Positif    226
Negatif     32
Name: Sentimen, dtype: int64

In [5]:
#Mengecek informasi pada kolom dataframe
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  500 non-null    int64 
 1   Tanggal     500 non-null    object
 2   Pengguna    500 non-null    object
 3   Tweet       500 non-null    object
 4   Sentimen    500 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.7+ KB


In [6]:
#Membuat datagrame baru yang terdiri dari kolom tweet dan sentimen saja
df = pd.DataFrame(tweet_df[['Tweet', 'Sentimen']])

In [7]:
#Tahapan Cleansing dan casefolding

# Menampilkan semua tanda baca dalam bentuk string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
# Menghilangkan tanda baca dan karakter yang tidak diperlukan
def remove_punct(Tweets):
      #Menggabungkan karakter pada tweet
      Tweets = "".join([char for char in Tweets if char not in string.punctuation])
      #Menghilangkan angka
      Tweets = re.sub('[0-9]+', '', Tweets)
      #Menghilangkan url web
      Tweets = re.sub(r"http\S+", "", Tweets)
      Tweets = re.sub(r"www.\S+", "", Tweets)
      #Mengganti tweet ke huruf kecil  
      Tweets = Tweets.lower()
      return Tweets

# Menggunakan function pada kolom tweet
df['Tweet'] = df['Tweet'].apply(lambda x: remove_punct(x))
df.head()

,Tweet,Sentimen
0,yth kapolri densus bnpt stop mendiscredit islam dgn isis teroris radikal intoleransi karena buka...,Netral
1,bhabinkamtibmas polsek rawas ulu polres muratara polda sumsel menyebarkan maklumat kapolri tenta...,Netral
2,indahnya toleransi indonesia cc yth bapak menkopolhukam bapak kapolri bapak presiden saya percay...,Positif
3,resepsi kaesang erina berjalan sukses kapolri apresiasi masyarakat surakarta kapolri apresiasi m...,Positif
4,bhabinkamtibmas polsek rawas ulu polres muratara polda sumsel menyebarkan maklumat kapolri tenta...,Netral


In [9]:
#Menghilangkan duplikat pada tweet
df = df.reset_index(drop=True).drop_duplicates(subset='Tweet', keep="last", ignore_index=True)
df

,Tweet,Sentimen
0,yth kapolri densus bnpt stop mendiscredit islam dgn isis teroris radikal intoleransi karena buka...,Netral
1,indahnya toleransi indonesia cc yth bapak menkopolhukam bapak kapolri bapak presiden saya percay...,Positif
2,bhabinkamtibmas polsek rawas ulu polres muratara polda sumsel menyebarkan maklumat kapolri tenta...,Netral
3,kapolda sumsel beri penghargaan kepada personil yang berprestasi poldasumsel kapoldasumsel polri...,Netral
4,prosesi pernikahan kaesang dan erina berjalan sesuai rencana kapolri puji sinergi tni dan polri ...,Netral
...,...,...
293,polsek mesuji makmur ulurkan tangan mu untuk membantu sesama untuk memberikan manfaat bagi orang...,Netral
294,arahan bapak kapolri kepada jajaran untuk meraih kembali kepercayaan publik hingga menghindari p...,Netral
295,polsek mesuji makmur tebarkan senyuman dengan penuh ke ikhlasan dalam setiap langkah mabespolri ...,Positif
296,cega pak knp kinerja kapolri ini tdk di kontrol knp dbiarkan sj seenaknya spt itu tdk tg jwb di ...,Netral


In [21]:
#Menyimpan dataframe
df.to_csv("D:/hasilSentimen2/Data-hasil-cleansing-casefolding.csv",encoding='utf-8',index=False)

In [33]:
#Mengimpor kembali dataframe hasil cleansing diatas
data2 = pd.read_csv('D:/hasilSentimen2/Data-hasil-cleansing-casefolding.csv',encoding='ISO-8859-1')
data2

,Tweet,Sentimen
0,yth kapolri densus bnpt stop mendiscredit islam dgn isis teroris radikal intoleransi karena buka...,Negatif
1,indahnya toleransi indonesia cc yth bapak menkopolhukam bapak kapolri bapak presiden saya percay...,Positif
2,tuan presdent panglima tni tni kapolri lihatlah anak bangsamu menjadi korban keganasan opm,Negatif
3,terima kasih atas kerja keras kekompakan dan loyalitas tanpa batas sehingga kegiatan berjalan am...,Positif
4,calon kapolri thn kedepan,Positif
...,...,...
139,yth pak tolong rakyat kami tanahnya dirampas bahkan tdk boleh masuk dilarang satpam mohon keadil...,Negatif
140,pak pak kapolri bantu kami kemana lagi kami melapor mafialahan mafiasawit,Positif
141,mereka berani berkhianat karena polisi dan tni sudah bisa digunakan untuk menghajar rakyat yang ...,Negatif
142,mantab pak kapolri sigap berwibawa semoga acara pernikahan putra ketiga presiden kita berjalan l...,Positif


In [34]:
# Membuat fungsi tokenization untuk memisahkan kalimat menjadi kata
def tokenization(Tweets):
  Tweets = re.split('\W+', Tweets)
  return Tweets

#Menerapkan function pada kolom tweet
data2['Tweet'] = data2['Tweet'].apply(lambda x: tokenization(x.lower()))
#Menampilkan 50 data teratas
data2.head(50)

,Tweet,Sentimen
0,"[yth, kapolri, densus, bnpt, stop, mendiscredit, islam, dgn, isis, teroris, radikal, intoleransi...",Negatif
1,"[indahnya, toleransi, indonesia, cc, yth, bapak, menkopolhukam, bapak, kapolri, bapak, presiden,...",Positif
2,"[tuan, presdent, panglima, tni, tni, kapolri, lihatlah, anak, bangsamu, menjadi, korban, keganas...",Negatif
3,"[terima, kasih, atas, kerja, keras, kekompakan, dan, loyalitas, tanpa, batas, sehingga, kegiatan...",Positif
4,"[calon, kapolri, thn, kedepan]",Positif
5,"[teroris, itu, fakta, ada, korbannya, khususnya, natal, bahwa, dalam, perjalanannya, ada, sesuat...",Negatif
6,"[maaf, bapak, kapolri, apakah, sptl, itu, baru, muncul, setelah, adanya, sphp, kalau, di, mintai...",Positif
7,"[laporan, hasil, pemeriksaan, div, propam, polri, terkait, kasus, suap, tambang, ilegal, ke, sej...",Negatif
8,"[tak, hanya, tni, polri, kapolri, sebut, densus, antiteror, ikut, amankan, pernikahan, kaesang, ...",Positif
9,"[justru, yg, ga, tegas, adl, kapolri, amp, penegak, hukum, coba, klo, itu, pelakunya, orang, awa...",Negatif


In [25]:
#Menyimpan dataset hasil tokenizing
data2.to_csv("D:/hasilSentimen2/Data-hasil-tokenization.csv",encoding='utf-8',index=False)

In [26]:
#TAHAPAN MEMULAI NORMALISASI
#Normalisasi kata adalah sebuah tahapan untuk mengubah bentuk suatu kata yang tidak baku dalam suatu kalimat pada tweet menjadi suatu kata yang baku dalam suatu kalimat.

normalizad_word = pd.read_csv("D:/hasilSentimen2/new_kamusalay.csv", encoding='ISO-8859-1')

#Membuat dictionary
normalizad_word_dict = {}

#Melakukan perulangan dan pengecekan pada tiap baris dataset
for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 
        
#Melakukan normalisasi
def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

#Menggunakan function normalisasi ke kolom tweet
data2['Tweet'] = data2['Tweet'].apply(normalized_term)
#Menampilkan 100 data teratas
data2['Tweet'].head(100)

0     [yang terhormat, kepala kepolisian republik indonesia, betasemen khusus, badan nasional penanggu...
1     [indahnya, toleransi, indonesia, cc, yang terhormat, bapak, menkopolhukam, bapak, kepala kepolis...
2     [tuan, presdent, panglima, tentara nasional indonesia, tentara nasional indonesia, kepala kepoli...
3     [terima, kasih, atas, kerja, keras, kekompakan, dan, loyalitas, tanpa, batas, sehingga, kegiatan...
4                                          [calon, kepala kepolisian republik indonesia, tahun, ke depan]
                                                     ...                                                 
95     [harus, reformasi, total, pekerjaan rumah, berat, buat, pak, kepala kepolisian republik indonesia]
96                                             [kepala kepolisian republik indonesia, juga, komunis, itu]
97           [hehe, lihat, laganya, dia, besar, nanti, cocok, jadi, kepala kepolisian republik indonesia]
98    [mari, umat, islam, kita, doa, bersama, 

In [28]:
#STOPWORD

#Membuat fungsi stopword untuk menghilangkan kata yang tidak perlu /tidak ada makna
def stopwords(Tweets):
  reg = re.compile(r"\n")
  return reg.split(Tweets)

# Membuka file contoh stopword
file = open("D:/hasilSentimen2/stopwords.txt","r");
st = file.read()
stopwords = stopwords(st)
stopwords

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [29]:
# Membuat fungsi menghapus stopword untuk mengilangkan stopword pada tweet berdasarkan contoh diatas
def remove_stopwords(Tweets):
  Tweets = [word for word in Tweets if word not in stopwords]
  return Tweets

#Menerapkan fungsi pada tweet 
data2['Tweet'] = data2['Tweet'].apply(lambda x: remove_stopwords(x))
#Menampilkan 50 data teratas
data2.head(50)

,Tweet,Sentimen
0,"[yang terhormat, kepala kepolisian republik indonesia, betasemen khusus, badan nasional penanggu...",Negatif
1,"[indahnya, toleransi, indonesia, cc, yang terhormat, menkopolhukam, kepala kepolisian republik i...",Positif
2,"[tuan, presdent, panglima, tentara nasional indonesia, tentara nasional indonesia, kepala kepoli...",Negatif
3,"[terima, kasih, kerja, keras, kekompakan, loyalitas, batas, kegiatan, berjalan, aman, lancar, li...",Positif
4,"[calon, kepala kepolisian republik indonesia, ke depan]",Positif
5,"[teroris, fakta, korbannya, natal, perjalanannya, pimpinan redaksi, serra, merta, menggampangkan...",Negatif
6,"[maaf, kepala kepolisian republik indonesia, sptl, muncul, sphp, mintai, sptl, saksinya, interna...",Positif
7,"[laporan, hasil, pemeriksaan, div, propam, kepolisian republik indonesia, terkait, suap, tambang...",Negatif
8,"[tentara nasional indonesia, kepolisian republik indonesia, kepala kepolisian republik indonesia...",Positif
9,"[kepala kepolisian republik indonesia, amp, penegak, hukum, coba, pelakunya, orang, awam, dor, m...",Negatif


In [30]:
#Menghitung kembali jumlah sentimen
data2["Sentimen"].value_counts()

Negatif    87
Positif    57
Name: Sentimen, dtype: int64

In [31]:
#Menyimpan dataset
data2.to_csv("D:/hasilSentimen2/Data-hasil-stopword.csv",encoding='utf-8',index=False)